In [11]:
from sage.all import *

p = 1070960903638793793346073212977144745230649115077006408609822474051879875814028659881855169
A = 0
B = 19
E = EllipticCurve(GF(p),[A,B])

Px = 850194424131363838588909772639181716366575918001556629491986206564277588835368712774900915
Py = 749509706400667976882772182663506383952119723848300900481860146956631278026417920626334886

Qx = 54250358642669756154015134950152636682437522715786363311759940981383592083045988845753867
Qy = 324772290891069325219931358863917293864610371020855881775477694333357303867104131696431188

P = E(Px, Py)
Q = E(Qx, Qy)
def SmartAttack(P,Q,p):
    E = P.curve()
    Eqp = EllipticCurve(Qp(p, 2), [ ZZ(t) + randint(0,p)*p for t in E.a_invariants() ])

    P_Qps = Eqp.lift_x(ZZ(P.xy()[0]), all=True)
    for P_Qp in P_Qps:
        if GF(p)(P_Qp.xy()[1]) == P.xy()[1]:
            break

    Q_Qps = Eqp.lift_x(ZZ(Q.xy()[0]), all=True)
    for Q_Qp in Q_Qps:
        if GF(p)(Q_Qp.xy()[1]) == Q.xy()[1]:
            break

    p_times_P = p*P_Qp
    p_times_Q = p*Q_Qp

    x_P,y_P = p_times_P.xy()
    x_Q,y_Q = p_times_Q.xy()

    phi_P = -(x_P/y_P)
    phi_Q = -(x_Q/y_Q)
    k = phi_Q/phi_P
    return ZZ(k)

d = SmartAttack(P, Q, p)
print(d)

import hashlib
from Crypto.Cipher import AES, DES
from Crypto.Util.Padding import unpad
from Crypto.Util.number import long_to_bytes

d_found = d  # <--- 在这里填入上面算出的 d

# 题目密文参数
aes_iv_hex = "7d0e47bb8d111b626f0e17be5a761a14"
des_iv_hex = "86fd0c44751700d4"
ciphertext_hex = (
    "7d34910bca6f505e638ed22f412dbf1b50d03243b739de0090d07fb097ec0a2c"
    "a19158949f32e39cd84adea33d2229556f635237088316d2"
)

def decrypt():
    if d_found == 0:
        print("[-] 请先运行 SageMath 脚本算出 d，并填入 d_found 变量")
        return

    # 1. 还原密钥
    k = long_to_bytes(d_found)
    aes_key = hashlib.sha256(k + b"MeOwl::AES").digest()[:16]
    des_key = hashlib.sha256(k + b"MeOwl::DES").digest()[:8]

    ciphertext = bytes.fromhex(ciphertext_hex)
    aes_iv = bytes.fromhex(aes_iv_hex)
    des_iv = bytes.fromhex(des_iv_hex)

    try:
        # 2. 解密外层 (DES)
        # DES 使用 pad(c1, 8)，解密后需 unpad 8
        des_cipher = DES.new(des_key, DES.MODE_CBC, iv=des_iv)
        c1_padded = des_cipher.decrypt(ciphertext)
        c1 = unpad(c1_padded, 8)

        # 3. 解密内层 (AES)
        # AES 使用 pad(flag, 16)，解密后需 unpad 16
        aes_cipher = AES.new(aes_key, AES.MODE_CBC, iv=aes_iv)
        flag_padded = aes_cipher.decrypt(c1)
        flag = unpad(flag_padded, 16)

        print(f"[+] Flag: {flag.decode()}")
        
    except Exception as e:
        print(f"[-] Decryption failed: {e}")

if __name__ == "__main__":
    decrypt()

797362141196384868007066615792142575269512834174354422840008233995912822094171106431346799
[+] Flag: 0xfun{n0n_c4n0n1c4l_l1f7s_r_c00l}
